In [24]:
import pandas as pd
import numpy as np
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, r2_score
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [25]:
banka = pd.read_csv("C:\Hogwarts\machine_learning\Cases\Bankruptcy\Bankruptcy.csv",
                   index_col = 0)

In [26]:
banka.head()

,D,YR,R1,R2,R3,R4,R5,R6,R7,R8,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
NO,,,,,,,,,,,,,,,,,,,,,
1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,10.36,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,3.13,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,2.41,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,5.55,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,2.85,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06


In [27]:
X = banka.drop(['D', 'YR'] , axis =1)
y = banka['D']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify = y,
                                                    random_state = 2022,
                                                    train_size = 0.7)

In [38]:
lr  = LogisticRegression()
nb  = GaussianNB()
da = LinearDiscriminantAnalysis()
rf  = RandomForestClassifier()
stack = StackingClassifier([('LR' , lr),
                            ('NB' , nb),
                            ('DA' , da)],
                           final_estimator= rf,
                          passthrough = True)


In [39]:
stack.fit(X_train , y_train)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

StackingClassifier(estimators=[('LR', LogisticRegression()),
                               ('NB', GaussianNB()),
                               ('DA', LinearDiscriminantAnalysis())],
                   final_estimator=RandomForestClassifier(), passthrough=True)

In [42]:
y_pred  = stack.predict(X_test)

In [43]:
print(accuracy_score(y_test , y_pred))

0.9


In [46]:
y_pred_prob = stack.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test , y_pred))

0.9


In [47]:
kfold = StratifiedKFold(n_splits = 5,
                       shuffle = True,
                       random_state = 2022)

In [49]:
print(stack.get_params())

{'cv': None, 'estimators': [('LR', LogisticRegression()), ('NB', GaussianNB()), ('DA', LinearDiscriminantAnalysis())], 'final_estimator__bootstrap': True, 'final_estimator__ccp_alpha': 0.0, 'final_estimator__class_weight': None, 'final_estimator__criterion': 'gini', 'final_estimator__max_depth': None, 'final_estimator__max_features': 'sqrt', 'final_estimator__max_leaf_nodes': None, 'final_estimator__max_samples': None, 'final_estimator__min_impurity_decrease': 0.0, 'final_estimator__min_samples_leaf': 1, 'final_estimator__min_samples_split': 2, 'final_estimator__min_weight_fraction_leaf': 0.0, 'final_estimator__n_estimators': 100, 'final_estimator__n_jobs': None, 'final_estimator__oob_score': False, 'final_estimator__random_state': None, 'final_estimator__verbose': 0, 'final_estimator__warm_start': False, 'final_estimator': RandomForestClassifier(), 'n_jobs': None, 'passthrough': True, 'stack_method': 'auto', 'verbose': 0, 'LR': LogisticRegression(), 'NB': GaussianNB(), 'DA': LinearDis

In [50]:
params= {"LR__C": np.linspace(0,5,5),
         "final_estimator__max_features":[2,4,6,8]}

In [51]:
gcv = GridSearchCV(stack , param_grid= params, cv = kfold ,
                   verbose  = 3 ,
                  scoring = "roc_auc")

In [52]:
gcv.fit(X , y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END LR__C=0.0, final_estimator__max_features=2;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.0, final_estimator__max_features=2;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.0, final_estimator__max_features=2;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.0, final_estimator__max_features=2;, score=nan total time=   0.0s
[CV 5/5] END LR__C=0.0, final_estimator__max_features=2;, score=nan total time=   0.0s
[CV 1/5] END LR__C=0.0, final_estimator__max_features=4;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.0, final_estimator__max_features=4;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.0, final_estimator__max_features=4;, score=nan total time=   0.0s
[CV 4/5] END LR__C=0.0, final_estimator__max_features=4;, score=nan total time=   0.0s
[CV 5/5] END LR__C=0.0, final_estimator__max_features=4;, score=nan total time=   0.0s
[CV 1/5] END LR__C=0.0, final_estimator__max_features=6;, score=nan 

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 1/5] END LR__C=1.25, final_estimator__max_features=2;, score=0.865 total time=   0.3s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 2/5] END LR__C=1.25, final_estimator__max_features=2;, score=0.912 total time=   0.3s


...

[CV 4/5] END LR__C=5.0, final_estimator__max_features=8;, score=0.979 total time=   0.3s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[CV 5/5] END LR__C=5.0, final_estimator__max_features=8;, score=0.994 total time=   0.3s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py", line 660, in fit
    return super().fit(X, y_encoded, sample_weight)
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\ensemble\_stacking.py", line 210, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  Fi

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=StackingClassifier(estimators=[('LR',
                                                       LogisticRegression()),
                                                      ('NB', GaussianNB()),
                                                      ('DA',
                                                       LinearDiscriminantAnalysis())],
                                          final_estimator=RandomForestClassifier(),
                                          passthrough=True),
             param_grid={'LR__C': array([0.  , 1.25, 2.5 , 3.75, 5.  ]),
                         'final_estimator__max_features': [2, 4, 6, 8]},
             scoring='roc_auc', verbose=3)

In [53]:
print(gcv.best_params_)

{'LR__C': 3.75, 'final_estimator__max_features': 4}


In [54]:
print(gcv.best_score_)

0.9266272189349112
